In [ ]:
#----Initialization----------------------
import numpy as np
import pandas as pd
import tensorflow
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import multivariate_normal
from scipy.special import softmax
from scipy import sparse
from scipy.optimize import leastsq
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import math
pi=math.pi
exp=math.exp

In [ ]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
inp_up=np.loadtxt('Gfupin.dat') # input with 2,4,6 orbital 100 random U
inp_dw=np.loadtxt('Gfdwin.dat')
out_up=np.loadtxt('Gfupout.dat') # output with 8 orbital 100
out_dw=np.loadtxt('Gfdwout.dat')
#------------------To cross check we reshape-------------
Ginp_up=np.reshape(inp_up, (500,256*3), order='C')
Ginp_dw=np.reshape(inp_dw, (500,256*3),order='C')
Gout_up=np.reshape(out_up, (500,256),order='C')
Gout_dw=np.reshape(out_dw, (500,256),order='C')

tmp=np.reshape(Ginp_up,(500,256,3))
tmpdw=np.reshape(Ginp_dw,(500,256,3))

interaction=np.loadtxt('../U.dat')
dfint=pd.DataFrame(data=interaction)
dfint.hist(bins=100, grid=False, figsize=(4,4), color='#86bf91', zorder=2, rwidth=0.9)
plt.xlabel('U')
plt.ylabel("units")
plt.show()

In [ ]:
plt.plot(tmp[66,:,1])
plt.xlabel(r'$\tau$')
plt.ylabel(r'$G(\tau)$')
plt.show()

In [ ]:
wn=np.arange(0,256)
x=wn
y=tmp[66,:,1]+0.0001

In [ ]:
# Funtion fit using least square
def func2(params, x, y):
    a,b,c,d,e=params[0], params[1], params[2],params[3], params[4]
    residual=y-(a*np.exp(b*x**2-c*x-d)+e)
    return residual

In [ ]:
params = [0, 0, 0, 0, 0]
result = leastsq(func2, params, (x, y),maxfev=20000)
a, b, c,d, e=result[0][0], result[0][1], result[0][2],result[0][3], result[0][4]
print(a,b,c,d,e)
yfit1=a*np.exp(b*x**2-c*x-d)+e

In [ ]:
x=wn
plt.plot(x, y, 'bo', label="y-original")
plt.plot(x, yfit1, color="red", label="y=a*exp(b*x^2-c*x-d)+e")
plt.ylabel('y')
plt.legend(loc='best', fancybox=True, shadow=True)
plt.grid(True)
plt.show()

In [ ]:
popt_f=[]
for i in range(500): 
    params = [0, 0, 0, 0, 0]
    ydata=tmp[i,:,0]+0.0001
    result = leastsq(func2, params, (wn, ydata),maxfev=20000)
    a, b, c, d, e=result[0][0], result[0][1], result[0][2],result[0][3], result[0][4]
    print(i)
    popt_f.append([a,b,c,d,e])

d2orb=pd.DataFrame(popt_f)   
d2orb.columns=['param02','param12','param22','param32','param42']

In [ ]:
display(d2orb)

In [ ]:
d2orb
#print(d2orb.describe())
d2orb.hist(column=["param02", "param12", "param22", "param32", "param42"], sharey=True, figsize=(8,8))
plt.show()

In [ ]:
from sklearn.preprocessing import MaxAbsScaler
trans = MaxAbsScaler()
data = trans.fit_transform(d2orb)
data2=pd.DataFrame(data)
data2.columns=['scaled_param02','scaled_param12','scaled_param22','scaled_param32','scaled_param42']
data2.hist(column=['scaled_param02','scaled_param12','scaled_param22','scaled_param32','scaled_param42'],sharey=True, figsize=(8,8))
plt.show()

In [ ]:
popt_f=[]
for i in range(500):
    params = [0, 0, 0, 0, 0]
    ydata=tmp[i,:,1]+0.001
    result = leastsq(func2, params, (wn, ydata), maxfev=20000)
    a, b, c,d, e=result[0][0], result[0][1], result[0][2],result[0][3], result[0][4]
    print(i)
    popt_f.append([a,b,c,d,e])
d4orb=pd.DataFrame(popt_f)
d4orb.columns=['param04','param14','param24','param34','param44']
d4orb

In [ ]:
d4orb
#print(d4orb.describe())
d4orb.hist(column=["param04", "param14", "param24", "param34", "param44"], sharey=True, figsize=(8,8))
plt.show()
from sklearn.preprocessing import MaxAbsScaler
trans = MaxAbsScaler()
data = trans.fit_transform(d4orb)
data4=pd.DataFrame(data)
data4.columns=['scaled_param04','scaled_param14','scaled_param24','scaled_param34','scaled_param44']
data4.hist(column=['scaled_param04','scaled_param14','scaled_param24','scaled_param34','scaled_param44'],sharey=True, figsize=(8,8))
plt.show()

In [ ]:
popt_f=[]
for i in range(500):
    params = [0, 0, 0, 0, 0]
    ydata=tmp[i,:,2]+0.0001
    result = leastsq(func2, params, (wn, ydata),maxfev=20000)
    a, b, c,d, e=result[0][0], result[0][1], result[0][2],result[0][3], result[0][4]
    print(i)
    popt_f.append([a,b,c,d,e])
d6orb=pd.DataFrame(popt_f)
d6orb.columns=['param06','param16','param26','param36','param46']
d6orb

In [ ]:
d6orb
#print(d6orb.describe())
d6orb.hist(column=["param06", "param16", "param26", "param36", "param46"], sharey=True, figsize=(8,8))
plt.show()
from sklearn.preprocessing import MaxAbsScaler
trans = MaxAbsScaler()
data = trans.fit_transform(d6orb)
data6=pd.DataFrame(data)
data6.columns=['scaled_param06','scaled_param16','scaled_param26','scaled_param36','scaled_param46']
data6.hist(column=['scaled_param06','scaled_param16','scaled_param26','scaled_param36','scaled_param46'],sharey=True, figsize=(8,8))
plt.show()

In [ ]:
popt_f=[]
for i in range(500):
    params = [0, 0, 0, 0, 0]
    ydata=Gout_up[i,:]+0.0001
    result = leastsq(func2, params, (wn, ydata),maxfev=20000)
    a, b, c,d, e=result[0][0], result[0][1], result[0][2],result[0][3], result[0][4]
    print(i)
    popt_f.append([a,b,c,d,e])
d8orb=pd.DataFrame(popt_f)
d8orb.columns=['param08','param18','param28','param38','param48']
d8orb.hist(column='param08',bins=20)

In [ ]:
d8orb
#print(d8orb.describe())
d8orb.hist(column=["param08", "param18", "param28", "param38", "param48"], sharey=True, figsize=(8,8))
plt.show()
from sklearn.preprocessing import MaxAbsScaler
trans = MaxAbsScaler()
data = trans.fit_transform(d8orb)
data8=pd.DataFrame(data)
data8.columns=['scaled_param08','scaled_param18','scaled_param28','scaled_param38','scaled_param48']
data8.hist(column=['scaled_param08','scaled_param18','scaled_param28','scaled_param38','scaled_param48'],sharey=True, figsize=(8,8))
plt.show()

In [ ]:
Gin = pd.concat([data2, data4,data6], axis=1)
Gin_t=pd.concat([d2orb, d4orb,d6orb], axis=1)
X=Gin.to_numpy()
X_t=Gin_t.to_numpy()
y=data8.to_numpy()
print(y.shape[0])

In [ ]:
print(X.shape, y.shape)
X_train=X[0:400,:]
X_test=X[400:500,:]
X_test_t=X_t[400:500,:]
y_train=y[0:400,:]
y_test=y[400:500,:]
print(y_train.shape)

In [ ]:
def wider_model():
    # create model
    model = Sequential()
    model.add(Dense(20, input_dim=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(5, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model
# evaluate model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasRegressor(build_fn=wider_model, epochs=10, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10)
results = cross_val_score(pipeline, X, y, cv=kfold)
print("Wider: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Gin, data8, test_size=0.1, random_state=0)
X_test.shape

In [ ]:
from keras.layers import GaussianNoise
from tensorflow.keras.regularizers import l2

values = [1e-4]
all_train, all_test = list(), list()
for params in values:
    model = Sequential()
    model.add(Dense(10, activation = 'relu',input_shape=(15,)))
    model.add(Dense(5, activation = 'relu'))
    model.add(Dense(5))
    #model.compile(loss = 'mse', optimizer = RMSprop(), metrics = ['accuracy'])
    model.compile(loss = 'mae', optimizer = 'adam', metrics = ['accuracy'])
    history = model.fit(X_train, y_train, batch_size=20, epochs = 2000, verbose = 1, validation_data=(X_test,y_test))
    _, train_acc = model.evaluate(X_train, y_train, verbose=0)
    _, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print('Param: %f, Train: %.3f, Test: %.3f' % (params, train_acc, test_acc))
    all_train.append(train_acc)
    all_test.append(test_acc)  

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.ylim(0.0,0.65)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.4f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
print(model.summary())

In [ ]:
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='valLoss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
prediction = model.predict(X_test) 
print(prediction.shape) 
ypred=prediction
print(ypred.shape)

In [ ]:
ypred
ypred_h=pd.DataFrame(ypred)
ypred_h.hist()
data8.iloc[400:500,:].hist()

In [ ]:
a=ypred[0][0]
b=ypred[0][1]
c=ypred[0][2]
d=ypred[0][3]
e=ypred[0][4]
print(a,b,c,d,e)

In [ ]:
yfit1=a*np.exp(b*x**2-c*x-d)+e

In [ ]:
plt.plot(x, yfit1, color="red", label="y=ax^2+bx+c")